# 📘 K-means Clustering of Air Pollution Time Series (R Tutorial)

### 🔍 Objective
Group spatial grid points based on their air pollution patterns over time using k-means clustering.

## 1. 📦 Load Required Libraries

In [ ]:
# Install if needed
install.packages("tidyverse")

In [ ]:
install.packages("reshape2")

In [ ]:
install.packages("ggplot2")

In [ ]:
install.packages("cluster")

In [ ]:
install.packages("factoextra")

In [ ]:
install.packages("lubridate")

In [ ]:
install.packages("sf")

In [ ]:
# Load necessary libraries
library(tidyverse)   # For data manipulation and visualization
library(reshape2)    # For reshaping data from long to wide format
library(ggplot2)     # For advanced plotting
library(cluster)     # For clustering algorithms
library(factoextra)  # For visualizing clustering diagnostics
library(lubridate)   # For date parsing and handling
library(sf)        # For reading shapefiles
#library(dplyr)

## 📂 2. Read athe dataset

Read the file from google drive

https://drive.google.com/file/d/FILE_ID/view?usp=sharing

You extract the FILE_ID part (the long string between /d/ and /view).

In [ ]:
FILE_ID = "FILE_ID"
url <- paste0("https://drive.google.com/uc?export=download&id=", FILE_ID)

# Read the file from google drives
data <- read.csv(url)

In [ ]:
dim(data)
data

In [ ]:
colnames(data)

In [ ]:
# Check for missing values
sum(is.na(data))          # returns 0 if there are no missing values

## 3. 🧹 Preprocess Data

We want to reshape the data so that each spatial grid point (lat/lon pair) is a row, and the time series of pollution is a feature vector.

In [ ]:
# Example of PM2.5
pollution_wide <- data %>%
  select(latitude, longitude, valid_time, pm2p5) %>%
  pivot_wider(names_from = valid_time, values_from = pm2p5)

# Remove columns (time steps) with NA values (if any time points are missing)
pollution_wide <- pollution_wide[, colSums(is.na(pollution_wide)) == 0]

In [ ]:
dim(pollution_wide)
head(pollution_wide)

## 4. 📐 Scale the Time Series

Normalize each time series (per grid point) to prevent K-means from grouping based on absolute levels instead of pattern

In [ ]:
# Extract time series only
pollution_ts <- pollution_wide %>% select(-latitude, -longitude)

# Standardize each time series
pollution_scaled <- scale(pollution_ts)

In [ ]:
dim(pollution_scaled)
head(pollution_scaled)

## 5. 🔢 Optional: Determine Optimal Number of Clusters

### Use Elbow Method to Choose k
**Elbow Method** minimizes the within-cluster sum of squares (WCSS) — this measures how close the data points in each cluster are to the cluster center (centroid).

In [ ]:
fviz_nbclust(pollution_scaled, kmeans, method = "wss", k.max = 20) +
  labs(title = "Elbow Method to Determine Optimal k")


### Use Silhouette Method to Choose k

**Silhouette Method** measure how well-separated the clusters are — how similar a point is to its own cluster vs. others.

In [ ]:
# Silhouette Method
fviz_nbclust(pollution_scaled, kmeans, method = "silhouette", k.max = 20)


## 6. 📌 Run K-means Clustering

In [ ]:
# Run k-means
set.seed(123)

k <- 13  # or choose based on the elbow plot
kmeans_result <- kmeans(pollution_scaled, centers = k, nstart = 25)

"*nstart*" controls how many times the algorithm runs with different random initial centroids,

## 7. 📈 View Clustering Results

In [ ]:
kmeans_result

In [ ]:
kmeans_result$cluster

In [ ]:
# Add cluster labels
pollution_wide$cluster <- as.factor(kmeans_result$cluster)

In [ ]:
dim(pollution_wide)
pollution_wide

## 8. 🗺️ Visualize Clusters on a Map

In [ ]:
# Read shapefile
shapefile_path <- "/content/World_Countries__Generalized_.shp"
shape_morocco <- st_read(shapefile_path)

In [ ]:
# If your pollution data and shapefile are in different coordinate reference systems (CRS), make sure to match them

st_crs(shape_morocco)

In [ ]:
# If needed:
shape_morocco <- st_transform(shape_morocco, crs = 4326)  # WGS84

In [ ]:
# Compute bounding box from data
lon_range <- range(pollution_wide$longitude, na.rm = TRUE)
lat_range <- range(pollution_wide$latitude, na.rm = TRUE)


# Plot your clustered pollution map with shapefile overlay
ggplot() +
  # Pollution clusters (raster-like tile plot)
  geom_tile(data = pollution_wide, aes(x = longitude, y = latitude, fill = factor(cluster)), color = "white") +

  # Overlay shapefile boundaries
  geom_sf(data = shape_morocco, fill = NA, color = "black", size = 0.5) +

  coord_sf(xlim = lon_range, ylim = lat_range, expand = FALSE) +
  labs(title = "Clusters of Air Pollution Patterns", fill = "Cluster") +
  theme_minimal()


In [ ]:
# Compute the Average PM10 per Location

avg_pm10 <- data %>%
  group_by(longitude, latitude) %>%
  summarise(PM10_avg = mean(pm2p5, na.rm = TRUE)) %>%
  ungroup()


# Plot the Average PM10 on a Map

ggplot() +
  geom_tile(data = avg_pm10, aes(x = longitude, y = latitude, fill = PM10_avg)) +
  geom_sf(data = shape_morocco, fill = NA, color = "black", size = 0.3) +
  scale_fill_gradient(
    name = "Avg PM2.5",
    low = "#FFE5E5",    # light red for low values
    high = "#8B0000"    # dark red (firebrick) for high values
  ) +
  coord_sf(xlim = lon_range, ylim = lat_range, expand = FALSE) +
  labs(
    title = "Average PM2.5 Concentration",
    x = "Longitude",
    y = "Latitude"
  ) +
  theme_minimal()
